# get reviews

In [1]:
import requests, time, json, re
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
# Disable chardet debug messages
import logging
logging.getLogger().setLevel(logging.WARNING)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from metadata import *

import multiprocessing
cpu_count = multiprocessing.cpu_count()

import os
os.getcwd()

'/home/jupyter/movie-analysis'

In [2]:
!pip install pandarallel -qq

In [3]:
from pandarallel import pandarallel
print(cpu_count)
pandarallel.initialize(nb_workers=cpu_count-1, progress_bar=True, use_memory_fs=False)

16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## check num of reviews

In [4]:
def get_all(row):
    film = row.film
    return movieReview(film).get_num_reviews()

### KR

In [5]:
kr=pd.read_pickle('data/kr_meta.pkl')

In [6]:
%%time
kr['num_reviews'] = kr.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 13.4 s, sys: 3.68 s, total: 17.1 s
Wall time: 3min 41s


In [7]:
kr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11651 entries, 0 to 11650
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         11651 non-null  object 
 1   rate         1536 non-null   float64
 2   meta         11651 non-null  object 
 3   num_reviews  11651 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 364.2+ KB


In [9]:
kr.to_pickle('data/kr_meta.pkl')

### CN

In [10]:
cn=pd.read_pickle('data/cn_meta.pkl')

In [11]:
%%time
cn['num_reviews'] = cn.parallel_apply(get_all, axis=1)

CPU times: user 31.9 s, sys: 8.57 s, total: 40.4 s
Wall time: 8min 56s


In [12]:
cn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28304 entries, 0 to 28303
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         28304 non-null  object 
 1   rate         1691 non-null   float64
 2   meta         28304 non-null  object 
 3   num_reviews  28304 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 884.6+ KB


In [13]:
cn.to_pickle('data/cn_meta.pkl')

### JP

In [14]:
jp=pd.read_pickle('data/jp_meta.pkl')

In [15]:
%%time
jp['num_reviews'] = jp.parallel_apply(get_all, axis=1)

Sleep...
Sleep...
CPU times: user 39.8 s, sys: 11.1 s, total: 50.8 s
Wall time: 10min 19s


In [16]:
jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34689 entries, 0 to 34688
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   film         34689 non-null  object 
 1   rate         4005 non-null   float64
 2   meta         34689 non-null  object 
 3   num_reviews  34689 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
jp.to_pickle('data/jp_meta.pkl')

## get detailed reviews for non-zero films

In [ ]:
kr=pd.read_pickle('data/kr_meta.pkl')
kr_have_reviews = kr[kr.num_reviews > 0].reset_index()
len(kr_have_reviews)

In [ ]:
%%time
num_review_lst = []
pbar = tqdm(range(0,len(kr_have_reviews)))
for line in pbar:
    pbar.set_description(f'get {line} of {len(kr_have_reviews)}')
    num_review_lst.append(movieReview(kr_have_reviews['film'][line]).get_num_reviews())
len(num_review_lst)

get 26 of 1536:   2% 26/1536 [00:09<08:42,  2.89it/s]

In [11]:
movieReview(kr_have_reviews['film'][1]).get_num_reviews()

72

In [89]:
class movieReview():
    def __init__(self, url):
        self.movie = url.split('/')[2]
        self.url = 'https://letterboxd.com/film/{}/reviews/'.format(self.movie)
        self.soup = get_soup(self.movie)
        
    def get_soup(self):
        page = ''
        while page == '':
            try:
                r = requests.get(self.url, verify = False, headers = headers)
                break
            except:
                print('Sleep...')
                time.sleep(5)
                continue
        soup = BeautifulSoup(r.content, 'html.parser')
        return soup
    
    def get_num_reviews(self):
        num_reviews_element = self.soup.find('a', href='/film/{}/reviews/'.format(self.movie))
        if num_reviews_element:
            try:
                num_reviews = num_reviews_element['title'].split('\xa0')[0]
                num_reviews = int(re.sub(',','',num_reviews))
                return num_reviews
            except:
                return 0
        else:
            return 0

In [91]:
movieReview(kr['film'][0]).get_num_reviews()

0

In [73]:
get_num_reviews(soup)

233861

In [11]:
reviews_lst = soup.find_all('li',class_='film-detail')
reviews_lst[0]

<li class="film-detail"> <a class="avatar -a40" href="/mshooster/"> <img alt="mshooster" height="40" src="https://secure.gravatar.com/avatar/b67ce42fd37865859e696ddace0ff71a?rating=PG&amp;size=80&amp;border=&amp;default=https%3A%2F%2Fs2.ltrbxd.com%2Fstatic%2Fimg%2Favatar80.74798e0d.png" width="40"/> </a> <div class="film-detail-content"> <div class="attribution-block"> <p class="attribution"> <span class="content-metadata"> <span class="date"> <a class="context" href="/mshooster/film/parasite-2019/"> Watched by <strong class="name">mshooster</strong> </a> <span class="_nobr">08 Apr 2019</span> </span> <span class="has-icon icon-16 icon-liked"></span> <a class="has-icon icon-comment icon-16 comment-count" href="/mshooster/film/parasite-2019/">1</a> </span> </p> </div> <div class="body-text -prose collapsible-text" data-full-text-url="/s/full-text/viewing:62775614/"> <p>I’m so exited!!!</p> </div> <p class="like-link-target react-component -monotone" data-component-class="globals.comps.L

In [67]:
def get_review(item):
    attri = item.find('p',class_='attribution')
    if attri:
        rate_date = {}
        for child in attri.children:
            if child.name == 'span':
                if child['class'][0] != 'rating':
                    user = child.find({'strong':'name'})
                    rate_date['user'] = user.text.strip() if user else None
                    date = child.find('span',class_='_nobr')
                    rate_date['date'] = date.text.strip() if date else None
                elif 'rating' in child['class']:
                    rate_date['rating'] = child['class'][-1].split('-')[-1]

        review = item.find('div',class_='body-text -prose collapsible-text')
        rate_date['review'] = review.text.strip() if review else None
        
    else:
        rate_date = None
    return rate_date

In [68]:
get_review(reviews_lst[0])

{'user': 'mshooster', 'date': '08 Apr 2019', 'review': 'I’m so exited!!!'}

In [69]:
get_review(reviews_lst[2])

{'rating': '10', 'user': 'Mr. M', 'date': '15 May 2019', 'review': 'Embargo?'}